In [6]:
import pandas as pd
from utils import *

In [2]:
df = pd.read_csv('./gnn/data/raw/gnn_data_dgl_train_small.csv')
df.head()

,id,sentence,eds,semlink,target_node,fn_frame,edge_targets,fn_roles
0,20816010,The number of stocks on the New York Stock Exc...,"{e3:\n e3:implicit_conj<0:160>{e SF prop, TENS...","{'id': 20816010, 'token_id': 12, 'vb_form': 'f...",e44,Change_position_on_a_scale,[],[]
1,21609003,The year-earlier figures included $9.9 million...,{e3:\n _1:_the_q<0:3>[BV x6]\n e9:_year-earlie...,"{'id': 21609003, 'token_id': 3, 'vb_form': 'in...",e3,NF,['x10'],['']
2,21820003,At the end of the term of the convertible loan...,"{e3:\n e7:focus_d<0:88>{e SF prop, TENSE unten...","{'id': 21820003, 'token_id': 22, 'vb_form': 'o...",e54,NF,"['x48', 'x55']","['', '']"
3,20475025,Eastern has publicly stated it is exceeding it...,{e3:\n _1:proper_q<0:7>[BV x6]\n x6:named<0:7>...,"{'id': 20475025, 'token_id': 7, 'vb_form': 'ex...",e20,NF,"['x15', 'x21']","['', '']"
4,21126003,"""This is an outstanding example of how the Eas...","{e3:\n e5:focus_d<0:240>{e SF prop, TENSE unte...","{'id': 21126003, 'token_id': 14, 'vb_form': 'w...",e26,IN,['x33'],['']


In [14]:
[y for x in list(df.apply(lambda x: string_of_list_to_list(x['fn_roles']), axis=1)) for y in x]

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Employee',
 '',
 '',
 'Cognizer',
 'Chosen',
 '',
 '',
 'Message',
 '',
 '',
 'Killer/Cause',
 '',
 '',
 '',
 '',
 'Perceiver_passive',
 'Phenomenon',
 'Agent',
 '',
 '',
 '',
 '',
 '',
 '',
 'Stimulus',
 'Experiencer',
 '',
 '',
 '',
 'Perceiver_passive',
 'Phenomenon',
 '',
 '',
 '',
 'Message',
 'Addressee',
 'Goods',
 'Text',
 '',
 '',
 '',
 'Buyer',
 'Goods',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Speaker',
 'Message',
 '',
 '',
 'Perceiver_passive',
 'Phenomenon',
 'Message',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Cause',
 'Effect',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Ingestor',
 '',
 'Manufacturer',
 'Product',
 '',
 '',
 'Recipient',
 'Theme',
 '',
 '',
 '',
 'Creator',
 'Created_entity',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Stimulus',
 'Experiencer',
 '',
 '',
 '',
 'Theme',
 'Cotheme',
 'Buyer',
 'Goods',
 '',
 '',
 '',
 '',
 '',
 '',

In [20]:
import dgl
from dgl.data import DGLDataset
import torch
import os

class KarateClubDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='karate_club')

    def process(self):
        nodes_data = pd.read_csv('./members.csv')
        edges_data = pd.read_csv('./interactions.csv')
        node_features = torch.from_numpy(nodes_data['Age'].to_numpy())
        node_labels = torch.from_numpy(nodes_data['Club'].astype('category').cat.codes.to_numpy())
        edge_features = torch.from_numpy(edges_data['Weight'].to_numpy())
        edges_src = torch.from_numpy(edges_data['Src'].to_numpy())
        edges_dst = torch.from_numpy(edges_data['Dst'].to_numpy())

        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=nodes_data.shape[0])
        self.graph.ndata['feat'] = node_features
        self.graph.ndata['label'] = node_labels
        self.graph.edata['weight'] = edge_features

        # If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True
        self.graph.ndata['train_mask'] = train_mask
        self.graph.ndata['val_mask'] = val_mask
        self.graph.ndata['test_mask'] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

dataset = KarateClubDataset()
graph = dataset

print(dataset)

Dataset("karate_club", num_graphs=1, save_path=/home/raymond/.dgl/karate_club)


In [17]:
torch.from_numpy(members['Club'].astype('category').cat.codes.to_numpy())

/tmp/ipykernel_28958/3308896075.py:1: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484806139/work/torch/csrc/utils/tensor_numpy.cpp:172.)
  torch.from_numpy(members['Club'].astype('category').cat.codes.to_numpy())


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int8)

In [8]:
cora_dataset = torch_geometric.datasets.Planetoid(root='./torchData', name="Cora")

Processing...
Done!


In [21]:
np.unique(cora_dataset[0].y.numpy())

array([0, 1, 2, 3, 4, 5, 6])

In [22]:
cora_dataset[0].train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [23]:
cora_dataset[0].y.shape

torch.Size([2708])